# 文本预处理

In [5]:
import collections
import re
from d2l import torch as d2l

In [4]:

#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 使用列表生成式生成一个列表
    # re.sub(): 正则表达式替换函数 
    # '[^A-Za-z]+': 正则表达式模式 [^...]: 表示"非"，匹配不在括号内的任何字符
    # A-Za-z: 表示所有英文字母（大小写）
    # +: 表示"一个或多个"前面的字符
    # .strip() 移除字符串两端的空白字符（空格、制表符、换行符等）
    # .lower() 将所有字母转换为小写形式
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
# 输出第一行
print(lines[0])
# 输出第十一行
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [14]:
# 进行词元化拆分
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])
tokens2 = tokenize(lines,"char")
print(tokens2[0])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's']


## 词表


In [8]:
# 建立词表类
class Vocab:  #@save
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # tokens 刚才生成的词元列表
        # reserved_tokens预定义的次元列表
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        # count_corpus函数统计每个词元在语料库中出现的次数，返回一个Counter对象
        counter = count_corpus(tokens)
        
        # 将Counter对象中的(词元, 频率)对按照频率降序排序，结果存储在self._token_freqs属性中。
        # counter.items(): 返回(token, frequency)元组列表
        # key=lambda x: x[1]: 按照频率进行排序(因为item返回的是(token, frequency)元组列表)
        # reverse=True: 降序排列(高频词元优先)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],reverse=True)
        
        # 未知词元的索引为0
        # 将特殊词元 <unk>放在列表的最前面 并将预定义的保留词元列表连接到特殊词元之后
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 使用字典递推式进行处理
        # enumerate函数： enumerate(self.idx_to_token) 会为列表中的每个元素生成一个(索引, 元素)对
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            # 删除其中一些出现频率很小的词元
            if freq < min_freq:
                break
            # 将符合条件的词元添加到两个列表中 并生成映射
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # 判断是否为单个词元
        if not isinstance(tokens, (list, tuple)):
            # 如果是单个词元 存在则返回对应的键值 不存在则返回0
            # dictionary.get(key, default_value)字典的内置参数 存在则返回key 不存在则返回default_value
            return self.token_to_idx.get(tokens, self.unk)
        # 通过列表生成式和递归操作逐个对词元进行处理
        return [self.__getitem__(token) for token in tokens]

    # 与上述方法同理 从id到token
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    # 统计词元频率
    # 对空列表和二维列表进行特使处理
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
        # 相当于
        # flattened_tokens = []
        # for line in tokens:
        #     for token in line:
        #         flattened_tokens.append(token)
        # tokens = flattened_tokens
    return collections.Counter(tokens)

In [ ]:
# count_corpus的使用
import collections

# 一维词元列表
tokens_1d = ['the', 'time', 'machine', 'by', 'h', 'g', 'wells', 'the', 'time', 'traveller']

# 统计词频
counter = count_corpus(tokens_1d)
print(counter)
# 输出：Counter({'the': 2, 'time': 2, 'machine': 1, 'by': 1, 'h': 1, 'g': 1, 'wells': 1, 'traveller': 1})

# 获取出现次数最多的3个词元
print(counter.most_common(3))
# 输出：[('the', 2), ('time', 2), ('machine', 1)]  (或其他频率相同的词元)

In [9]:
# 构建此表 打印高频词元及其索引
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [10]:
# 将文本转换成数字索引列表
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [12]:
# 整合上述功能 对时光机器文本进行处理
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    # tokens是一个二维列表 
    # [[字符1, 字符2, ...], [字符1, 字符2, ...], ...]
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

# corpus = []
# for line in tokens:      # 外层循环：遍历每一行
#     for token in line:   # 内层循环：遍历行中的每个词元
#         corpus.append(vocab[token])  # 将词元转换为索引并添加到结果列表
# 输入tokens: 二维列表，每行包含多个词元（字符） [['t', 'h', 'e'], ['t', 'i', 'm', 'e'], ...]
# 输出corpus: 一维列表，包含所有词元对应的索引[3, 4, 5, 3, 6, 7, 5, ...]
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)